# NLP RECSYS Eval Stage




In [1]:
import com.johnsnowlabs.nlp.SparkNLP
import com.johnsnowlabs.nlp.annotator._
import com.johnsnowlabs.nlp.base._
import org.apache.spark.ml.{Pipeline, PipelineModel, Transformer}
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions.{udf,to_timestamp}
import org.apache.spark.storage._
import org.apache.spark.ml.feature._
import org.apache.spark.ml.classification._
import org.apache.spark.ml.linalg.DenseVector
import org.apache.spark.ml.param.{Param, ParamMap}
import org.apache.spark.ml.util.{DefaultParamsReadable, DefaultParamsWritable, Identifiable}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{col, explode, udf}
import org.apache.spark.sql.types.{DataTypes, StructType}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator


val dataDir = sys.env("HOME") + "/recsys2020"
val dsName = "training1m"
val predName = "retweet.csv"

In [2]:
val df = spark.read.parquet(dataDir + s"/${dsName}.parquet")

val schema = new StructType()
    .add("tweet_id", StringType, true)
    .add("user_id", StringType, true)
    .add("out_pred", DoubleType, true)

val pred = spark.read.format("csv")
        .option("delimiter", ",")
        .schema(schema)
        .load(dataDir + s"/${dsName}.tsv")

In [3]:
pred

[tweet_id: string, user_id: string ... 1 more field]

In [4]:
df

[user_id: string, tweet_id: string ... 17 more fields]

In [5]:
val joined = pred.join(df, Seq("user_id", "tweet_id"), "outer")
joined

[user_id: string, tweet_id: string ... 18 more fields]

In [6]:
// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("has_retweet")
  .setPredictionCol("out_pred")
  .setMetricName("f1")
val f1 = evaluator.evaluate(joined)
println(s"F1 = ${f1}")

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 17, localhost, executor driver): scala.MatchError: [null,null] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator$$anonfun$1.apply(MulticlassClassificationEvaluator.scala:79)
	at org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator$$anonfun$1.apply(MulticlassClassificationEvaluator.scala:79)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:193)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:62)
	at org.apache.sp